# 부채의 현재가치와 펀딩 비율

이번 시간에는 미래 부채의 현재 가치를 계산하기 위해 부채를 할인하는 방법과 펀딩 비율을 측정하는 방법에 대해 살펴보겠습니다.

여기서 말하는 펀딩 비율이 가지는 의미는 자산의 현재 가치와 부채의 현재 가치 간의 비율입니다.

현재 가치를 계산하기 위해서는, 수익률 곡선으로부터 도출된 금리를 통해 미래의 부채를 현재가치로 할인해와야 합니다.

논의의 단순성을 위해, 현실적이지는 않지만 여기에서는 수익률 곡선이 평평하다고 가정합니다. 이렇게 되면 모든 만기구간에 대해 이자율은 동일해집니다.

각각의 시점 $t_i$에 만기가 도래하는 각각의 부채 $L_i$로 구성된 부채들의 전체 집합의 현재 가치는 다음과 같습니다.

$$ PV(L) = \sum_{i=1}^{k} B(t_i) L_i$$

여기서 $B(t_i)$는 $t_i$ 시점에 1 달러를 지급하는 할인채의 현재 가격입니다.

만약 수익률 곡선이 평평하다고 가정하면, 모든 만기 구간에 대해 금리는 $r$이 되며 이때의 $B(t)$는 다음과 같습니다.

$$B(t) = \frac{1}{(1+r)^t}$$



In [1]:
import numpy as np
import pandas as pd
import port_opt_toolkit as potk

%load_ext autoreload
%autoreload 2

In [3]:
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays $1 at time t where t is in years and r is the annual interest rate
    """
    return (1+r)**(-t)

In [4]:
b = discount(10, .03)
b

0.7440939148967249

만약 오늘 이 채권을 사서 연 3\%의 이자율로 10년 동안 보유하게 되면 만기시 $1를 받을 수 있음을 간단하게 확인할 수 있습니다.

In [5]:
b*(1.03**10)

1.0

In [6]:
def pv(l, r):
    """
    Compute the present value of a list of liabilities given by the time (as an index) and amounts
    """
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

1, 1.5, 2, 250만 달러의 4가지 부채가 있다고 가정해 보겠습니다. 이들 중 첫 번째는 3년 떨어져 있고 다음은 6개월 간격으로 떨어져 있다고 가정합니다. 이 때 연 3%의 금리를 기준으로 부채의 현재 가치를 계산해 보겠습니다.

개인 투자의 맥락에서 부채라는 것은 미래의 주택 구매 자금, 자녀의 대학 비용 또는 퇴직 이후 생활비 등과 같이 생애 중 주요 사건들을 대비하기 위한 저축과 같은 목표로 생각할 수 있습니다. 이러한 각각의 경우에는 미래의 어느 시점에 현금 흐름이 필요합니다. 그렇기 때문에 우리는 이를 부채로 생각할 수 있는 것입니다.

In [7]:
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5])

In [8]:
pv(liabilities, 0.03)

6.233320315080045

이제 우리는 현재의 자산 가치를 기반으로 펀딩 비율을 계산할 수 있습니다.

In [10]:
def funding_ratio(assets, liabilities, r):
    """
    Computes the funding ratio of a series of liabilities, based on an interest rate and current value of assets
    """
    return assets/pv(liabilities, r)

In [11]:
funding_ratio(5, liabilities, 0.03)

0.8021407126958777

만약 금리가 3%에서 2%로 내려간다면 펀딩 비율이 어떻게 바뀌는지 다시 계산해보겠습니다.

In [12]:
funding_ratio(5, liabilities, 0.02)

0.7720304366941648

금리와 펀딩 비율의 관계를 인터랙티브하게 표현해보겠습니다.

In [15]:
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64

In [16]:
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

def show_funding_ratio(assets, r):
    fr = funding_ratio(assets, liabilities, r)
    print(f'{fr*100:.2f}%')
    
controls = widgets.interactive(show_funding_ratio,
                                   assets=widgets.IntSlider(min=1, max=10, step=1, value=5),
                                   r=(0, .20, .01)
)
display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

위의 위젯을 통해 알 수 있듯이, 자산 가치가 하락하지 않더라도 자산 가치보다 펀딩 비율을 고려한다면 현금 보유는 위험한 선택이 될 수 있습니다. 현금은 자산가치가 떨어지지 않는다는 점에서는 '안전자산'이지만 만약 금리가 내려가면 부채 가치가 올라가기 때문에 현금은 매우 위험한 자산이 될 수 있습니다. 따라서 펀딩 비율(즉, 필요한 양과 비교하여 보유하고 있는 자금의 양) 측면에서 저축에 대해 생각한다면 현금은 위험한 자산이며 펀딩 비율이 감소할 수 있습니다.

`port_opt_toolkit.py` 파일에 `discount`, `pv` 및 `funding_ratio` 기능을 추가합니다.

```python
def discount(t, r):
    """
    Compute the price of a pure discount bond that pays a dollar at time t where t is in years and r is the annual interest rate
    """
    return (1+r)**(-t)

def pv(l, r):
    """
    Compute the present value of a list of liabilities given by the time (as an index) and amounts
    """
    dates = l.index
    discounts = discount(dates, r)
    return (discounts*l).sum()

def funding_ratio(assets, liabilities, r):
    """
    Computes the funding ratio of a series of liabilities, based on an interest rate and current value of assets
    """
    return assets/pv(liabilities, r)
```

다음 시간에는 현금 자산 보유의 문제점을 어떻게 해결할지에 대해 보다 자세히 다루어보겠습니다.